In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/equationsolver/model_final.json
/kaggle/input/equationsolver/test.jpeg
/kaggle/input/equationsolver/README.md
/kaggle/input/equationsolver/CNN test.ipynb
/kaggle/input/equationsolver/train_final.csv
/kaggle/input/equationsolver/model_final.h5
/kaggle/input/equationsolver/Data_extraction.ipynb


**IMPORTING NECESSARY LIBRARIES**

In [3]:
import pandas as pd
import numpy as np

np.random.seed(1212)

from keras.models import Model
from keras.layers import *
from keras import optimizers
from keras.layers import Input, Dense
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_data_format('channels_first')

**READING INPUT CSV FILE**

In [4]:
df_train = pd.read_csv('/kaggle/input/equationsolver/train_final.csv',index_col=False)
labels = df_train[['784']]

df_train.drop(df_train.columns[[784]],axis=1,inplace=True)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
labels = np.array(labels)

**CHANGING LABELS TO CATEGORICAL**

In [6]:
from keras.utils.np_utils import to_categorical
cat = to_categorical(labels,num_classes=13)
print(cat[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [7]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
l = []
for i in range(47504):
    l.append(np.array(df_train[i:i+1]).reshape(1,28,28))

In [9]:
np.random.seed(7)

**CREATING THE MODEL**

In [10]:
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(1 , 28, 28), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(13, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**LOADING PREVIOUSLY TRAINED MODEL**

In [11]:
from keras.models import model_from_json

In [12]:
json_file = open('/kaggle/input/equationsolver/model_final.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("/kaggle/input/equationsolver/model_final.h5")

**TESTING THE MODEL**

In [13]:
import cv2
import numpy as np
img = cv2.imread('/kaggle/input/equationsolver/test.jpeg', cv2.IMREAD_GRAYSCALE)
if img is not None:
    img = ~img
    ret,thresh = cv2.threshold(img, 127,255, cv2.THRESH_BINARY)
    ctrs, ret = cv2.findContours(thresh,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnt = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    w = int(28)
    h = int(28)
    train_data = []
    rects = []
    for c in cnt :
        x, y, w, h = cv2.boundingRect(c)
        rect = [x, y, w, h]
        rects.append(rect)
    bool_rect=[]
    for r in rects:
        l = []
        for rec in rects:
            flag = 0
            if rec != r:
                if r[0] < (rec[0] + rec[2] + 10) and rec[0] <(r[0] +r [2] + 10) and r[1] < (rec[1] + rec[3] + 10) and rec[1] <(r[1] + r[3] + 10):
                    flag = 1
                l.append(flag)
            if rec == r:
                l.append(0)
        bool_rect.append(l)
    dump_rect = []
    for i in range(0,len(cnt)):
        for j in range(0,len(cnt)):
            if bool_rect[i][j] == 1:
                area1 = rects[i][2]*rects[i][3]
                area2 = rects[j][2]*rects[j][3]
                if(area1 == min(area1,area2)):
                    dump_rect.append(rects[i])
    final_rect = [i for i in rects if i not in dump_rect]
    for r in final_rect:
        x = r[0]
        y = r[1]
        w = r[2]
        h = r[3]
        im_crop = thresh[y:y+h+10,x:x+w+10]
        im_resize = cv2.resize(im_crop, (28, 28))
        im_resize = np.reshape(im_resize, (1, 28, 28))
        train_data.append(im_resize)

In [14]:
s = ''
for i in range(len(train_data)):
    train_data[i] = np.array(train_data[i])
    train_data[i] = train_data[i].reshape(1,1,28,28)
    result = loaded_model.predict_classes(train_data[i])
    if(result[0]==10):
        s = s + '-'
    if(result[0]==11):
        s = s + '+'
    if(result[0]==12):
        s = s + '*'
    for j in range(10):
        if(result[0] == j):
            s = s + str(j)
            break
print(s)    

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


82+8


**EVALUATING THE EXPRESSION**

In [15]:
eval(s)

90